# Molecular simulations with Yaff: Advanced MD

This notebook is devoted to performing advanced molecular dynamics (MD) simulations in *Yaff*. As explained in the previous notebook, the general idea of MD simulations is to sample the molecular phase space in a smart way, by sampling those regions with high boltzmann probability. For the details of regular MD simulations, we refer to the previous notebook. 

Despite the success of regular MD simulations, their usage can be limited for real-life systems. Often, the phase space of these real-life systems contains enery barriers which can not be overcome during a regular MD simulation. Hence, several enhanced sampling methods were proposed to overcome this issue. Some advanced sampling methods exploit the constraints forced by the employed ensemble, such as **thermodynamic integration** in the $(N, V, \sigma_a = {\bf 0}, T)$ ensemble, wheras others use an additional bias potential, *i.e.* **umbrella sampling** and **metadynamics**. Herein, we explain how to these biased simulations can be performed in *Yaff*, and illustrate the importance of choosing good **collective variables**.

In this *Python* notebook, we will cover:

1. The system under study: Cyclohexane
2. The importance of collective variables
3. Metadynamics
4. Umbrella sampling

Note that these advanced simulations use additional *Python* modules, contained in `colvar.py` (for the collective variables), `DMTD.py` (for metadynamics), and `DUS.py` (for umbrella sampling). These *Python* packages are not yet avaialble in the newest *Yaff* release, but are provided in this folder.

## Chapter 1: The system under study: Cyclohexane

In this notebook, we study the cyclohexane molecule. This cyclic organic molecule has several **(meta)stable** conformations, called the **chair**, **(half)chair**, **boat**, and **inversed chair conformations**. The figure below shows the chair, boat and inversed chair conformation.

![alt text](chair2invchair.png "Cyclohexane conformations")

Herein, we aim to construct a free energy profile that is able to describe these three conformations, as well as the transformations from the chair to the inversed chair conformation via the boat conformation. Before going into the advanced simulation techniques, it is intructive to study the behavior of cyclohexane during a regular MD simulation. To this end, we run three MD simulations:

1. The first simulation starts from the chair conformation
2. The second simulation starts from the boat conformation
3. The third simulation starts from the inverse chair conformation

First, we run the MD simulation starting from the chair conformation:

In [ ]:
import os
import h5py
from yaff import *
import numpy as np

# MD input
number_of_md_steps = 10000
temp = 300*kelvin
timestep = 0.5*femtosecond

# Create system and force field objects
chair = System.from_file('chair.chk')
ff = ForceField.generate(chair, 'pars.txt', rcut=15*angstrom, alpha_scale=3.2, gcut_scale=1.5, smooth_ei=True)

# Initialize the thermostat and the screen logger
thermo = NHCThermostat(temp, timecon = 100.0*femtosecond)
vsl = VerletScreenLog(step=100)

# Initialize the input/output files
xyz = XYZWriter('traj_chair.xyz')
with h5py.File('traj_chair.h5', mode='w') as f:
    hdf = HDF5Writer(f, step=10)
    # Run the MD simulation
    md = VerletIntegrator(ff, timestep, hooks = [thermo, vsl, hdf, xyz])
    md.run(number_of_md_steps)

Visualize the output using [VMD](http://www.ks.uiuc.edu/Research/vmd/). To this end, execute

    vmd traj_chair.xyz

from a terminal window opened in the folder containing this notebook. Do you observe any transformations?

Now, run the second and the third simulation, corresponding to the boat and inverse chair conformation, respectively:

In [ ]:
# Create system and force field objects
boat = System.from_file('boat.chk')
ff = ForceField.generate(boat, 'pars.txt', rcut=15*angstrom, alpha_scale=3.2, gcut_scale=1.5, smooth_ei=True)

# Initialize the thermostat and the screen logger
thermo = NHCThermostat(temp, timecon=100.0*femtosecond)
vsl = VerletScreenLog(step=100)

# Initialize the input/output files
xyz = XYZWriter('traj_boat.xyz')
with h5py.File('traj_boat.h5', mode='w') as f:
    hdf=HDF5Writer(f, step=10)
    # Run the MD simulation
    md = VerletIntegrator(ff, timestep, hooks = [thermo, vsl, hdf, xyz])
    md.run(number_of_md_steps)

In [ ]:
# Create system and force field objects
ichair = System.from_file('ichair.chk')
ff = ForceField.generate(ichair, 'pars.txt', rcut=15*angstrom, alpha_scale=3.2, gcut_scale=1.5, smooth_ei=True)

# Initialize the thermostat and the screen logger
thermo = NHCThermostat(temp, timecon=100.0*femtosecond)
vsl = VerletScreenLog(step=100)

# Initialize the input/output files
xyz = XYZWriter('traj_ichair.xyz')
with h5py.File('traj_ichair.h5', mode='w') as f:
    hdf=HDF5Writer(f, step=10)
    # Run the MD simulation
    md = VerletIntegrator(ff, timestep, hooks = [thermo, vsl, hdf, xyz])
    md.run(number_of_md_steps)

Again visualize the output. Do you observe any transformations?

Indeed, as stated earlier, the barriers between the three conformations prohibit any transition. We need to enhance our sampling.

<br/>

## Chapter 2: The importance of collective variables

When we enhance the sampling of the MD simulations, we do this in those directions for which the sampling is limited due to the presence of **reaction barriers**. Those directions are described by a set of **collective variables**, adopting real values that indicate the progress along these directions.

Several options can be considered as a collective variable, such as distances between two molecules, angles between three molecules, etc. In essence, the collective variable should clearly describe the transition from one conformation to the other and adopt clearly separated values for each (meta)stable state. In mathematical terms, the collective variable should behave as a monotonically increasing function of transition progress.

In *Yaff*, the `colvar.py` library contains a set of predefined collective variables. To run enhanced sampling simulations, this library is imported first and subsequently, a collective variable is defined. For the system under study, cyclohexane, the dihedral angle of four subsequent carbon atoms in the ring seems a decent choice for the collective variable.

In [ ]:
import colvar

# Define the collective variable type ("DIHEDRAL", "ANGLE", "DISTANCE")
type = "DIHEDRAL"

# Define the atom numbers for which the collective variable is defined
n1 = 0
n2 = 13
n3 = 10
n4 = 7

# Instantiate a collective variable object
cv = colvar.Dihedral('dihedral_colvar', [n1,n2,n3,n4])

# Define the unit of our collective variable (useful for post-processing)
unit = deg

Let's compare the value of the dihedral during the MD simulations we performed earlier. Make a plot of the trajectory and the corresponding histogram. Reflect on the outcome. What do the axes represent?

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt

# The names of the different conformations
conf = ['chair', 'boat', 'ichair']

# Define the colors that will be used
colors = ['royalblue', 'crimson', 'darkseagreen']

# Create a stacked plot, 
f, ax = plt.subplots(2,1)

# For all of the three conformations:
for ic,c in enumerate(conf):
    # Define the output file
    f = 'traj_' + c + '.h5'
    # Create a time series containing the values of the collective variable
    t = cv.post(f)/unit
    # Plot this time series in the upper pane
    ax[0].plot(t, color=colors[ic])
    # Create a histogram from this time series
    h, b = np.histogram(t, density=True)
    # Plot this histogram
    ax[1].plot(b[:-1], h, color=colors[ic])

# Finally, show the figure
plt.show()

From the plots above, it is clear that no transformation between any of the three conformations was observed. In what follows, we will force such a transition by using either metadynamics or umbrella sampling.

<br/>

## Chapter 3: Metadynamics

A first example of an advanced sampling simulation is **metadynamics**. In this method, a bias potential is added to the potential energy surface at already frequented locations, forcing the system to visit states that have not been visited before. This bias potential $U_b$ is a function of the collective variable $q$ and often consists of a summation of Gaussian contributions with heights $h$ and widths $w$:

$$ U_b(q) = h \sum_{i=0}^{N(t)} \exp\left(-\frac{(q-q_i)^2}{2w^2}\right) $$

The bias potential is updated over time and each new Gaussian contribution centers around the most visited state of the MD steps since the last update of the bias potential, $q_i$. The latter update fashion should be maintained until the bias potential counteracts the underlying―unknown―free energy profile and each state becomes equally probable. Upon convergence, we hence find the free energy profile $F(q)$ as the negative of the bias potential: $F(q)=-U_b(q)$. For more information on metadynamics, look into following [reference](http://www.pnas.org/content/99/20/12562).

Metadynamics simulations can be run in *Yaff* via the additional `DMTD.py` module. 

In [ ]:
import DMTD

# Create system and force field objects
chair = System.from_file('chair.chk')
ff = ForceField.generate(chair, 'pars.txt', rcut=15*angstrom, alpha_scale=3.2, gcut_scale=1.5, smooth_ei=True)

# Initialize the thermostat and the screen logger
thermo = NHCThermostat(temp, timecon=100.0*femtosecond)
vsl = VerletScreenLog(step=100)

#Information on the Gaussian contributions
# The width
w = [15*deg]
# The height
h = 1*kjmol
# The collective variable
cv = colvar.Dihedral('dihedral4C', [n1, n2, n3, n4])
# Putting it all together
hill = [DMTD.Hills([cv], width=w, height=h)]

# Information on the MTD protocol:
# How often is the bias potential updated
n_updates = 100
# Number of MD steps before updating the bias
n_steps_per_update = 1000
# Note: the total simulation time is n_updates*n_step_per_update

# Initialize the input/output files
xyz = XYZWriter('traj_mtd.xyz', step=100)
with h5py.File('traj_mtd.h5', mode='w') as f:
    hdf = HDF5Writer(f,step=100)
    # Run the metadynamics MD simulation
    replica = DMTD.Metadynamics1D(ff, timestep, n_updates, n_steps_per_update, hill, hooks=[hdf,thermo,vsl,xyz])
    replica.runMeta()

The HDF5 file `traj_mtd.h5` contains all information on the bias potential, which is stored under the `trajectory/mtd` group. We can reconstruct the free energy from this file with the code below. Note that the minimum of this profile is (arbitrarily) shifted to 0 kJ/mol. Can you rationalize why we can make this shift? 

In [ ]:
# Read in the last row in the trajectory/mtd group,
# containing the all values of the collective variable
# at which a Gaussian contribution was added
with h5py.File('traj_mtd.h5', mode='r') as f1:
    positions = f1['trajectory/mtd'][-1,:]

# Construct a uniform grid along the collective variable [-60°, 60°]
# and initialize the corresponding free energy to zero
grid = np.arange(-60, 61, 1)*unit
free_energy = np.zeros(len(grid[:]))

# Reconstruct the free energy profile on this grid,
# using the known positions of the Gaussian contributions
for i in range(len(grid)):
    curr_pos = grid[i]
    # For each grid point, sum the contributions of the N Gaussians
    free_energy[i] = -(np.sum(h*np.exp(-(curr_pos-positions[:])**2/(2.*w[0]**2))))/kjmol

# Plot and show the figure
plt.plot(grid/unit, free_energy - np.min(free_energy))
plt.show()

Can you interpret this free energy profile? Which is the most stable state, which state is unstable? Moreover, how do you relate this with the previous results of the MD simulations? What about the accuracy of this free energy profile? Do you think this profile is converged? How will the parameter choice ($h$, $w$, number of steps) influence our result?

<br/>

## Chapter 4: Umbrella sampling

The second advanced simulation method we consider in this notebook is *umbrella sampling*. To allow the system to sample different parts of the phase space, we create a harmonic bias potential $U_{bias}$ as a function of the collective variable $q$ with spring constant $k$:

$$ U_{bias} = \frac{k}{2} (q-q_0)^2 $$

With the aid of this bias potential, the system will sample the phase space in that region centered around the point where the collective variable takes the value $q_0$.

By performing a number of umbrella sampling simulations in parallel, with different minima $q_0$, the entire phase space can be sampled. A free energy profile can subsequently be created from the **weighted histogram analysis method** (WHAM), which combines the essential information from the different simulations. More information on WHAM can be found in section 7.3.1 of *Understanding Molecular Simulations: From Algorithm to Application*.

To add a harmonic umbrella potential to the potential energy surface in *Yaff*, the additional module `DUS.py` needs to be imported. Based on this module, we can run simulation with different potentials (note that such simulations can be run in parallel if the architecture allows for it).

Below, an umbrella sampling simulation for the cyclohexane molecule is started with umbrellas centered along the values in [-55°,55°] with a spacing of 5°.

In [ ]:
import DUS

# The collective variable
cv = colvar.Dihedral('dihedral4C', [n1, n2, n3, n4])
# Construct the grid along which umbrellas will be placed
cvs = np.arange(-55, 60, 5)

# MD input equal for all umbrellas
number_of_md_steps = 2000
# Define the spring constant k
kappa = 0.25*kjmol/unit**2
# Define the temperature
temp = 300.*kelvin
# Define the timestep
timestep = 0.5*femtosecond

# Create a designated directory for the US simulation (if not already present)
if not os.path.exists('US'):
    os.mkdir('US')

# For each umbrella:
for c in cvs:
    # Make a folder for this umbrella (if not yet present)
    direc='US/US_'+str(c)+'/'
    if not os.path.exists(direc):
        os.mkdir(direc)
    
    # Create a System object (a .chk file has already been provided)
    system = System.from_file('structures/structure_' + str(c) + '.chk')
    
    # Create a force field object
    ff = ForceField.generate(system, 'pars.txt', rcut=15*angstrom, alpha_scale=3.2, gcut_scale=1.5, smooth_ei=True)

    # Initialize the thermostat and the screen logger
    thermo = NHCThermostat(temp, timecon=100.0*femtosecond)
    vsl = VerletScreenLog(step=100)
    
    # Initialize the input/output files:
    xyz = XYZWriter(direc + 'traj.xyz', step=10)
    with h5py.File(direc + 'traj.h5', mode='w') as f:
        hdf = HDF5Writer(f, step=10)
        # Create the umbrella at position c
        umbrellas = DUS.Umbrella([cv], [kappa], [c*unit])     
        # Initialize the US simulation
        replica = DUS.US(ff, timestep, number_of_md_steps, umbrellas, hooks=[hdf,thermo,xyz,vsl])
        # Run the simulation
        replica.runU()

We start by visualizing the collective variable trajectory for all simulations. In order to obtain a decent free energy profile, these trajectories should overlap sufficiently. 

In [ ]:
# Open a file that will be completed with relevant output
with open('metadata','w') as g:
    # Run over the different positions for the umbrella
    for i,c in enumerate(cvs):
        # Define the directory where the results are stored
        direc = 'US/US_' + str(c)
        # Define the HDF5 file containing the output
        f = direc + '/traj.h5'
        # Try to access the file (will fail if a simulation failed)
        try:
            # Return the values of the collective variable in this simulation
            cv_ser = cv.post(f)/unit
            # Plot these values
            plt.plot(cv_ser)
            # Create a 1D time series of the collective variable
            cv_ser = np.array(cv_ser).reshape(-1,1)
            t = np.arange(0,len(cv_ser)).reshape(-1,1)
            # Save this time series in a file named colvar
            np.savetxt('US/US_' + str(c) + '/colvar', np.hstack((t, cv_ser)))
            # Write the value of the collective variable and the harmonic spring constant
            g.write(direc + '/colvar\t' + str(c) + '\t' + str(kappa) + '\n')
        # If accessing one of the simulations fails, warn the user
        except IOError:
            print('One of your simulations failed')

# Plot the results
plt.show()

If the trajectories look decent (*i.e.*, have sufficient overlap), we can use the weighted histogram analysis method. We employ a WHAM script provided by the [Grossfield lab](http://membrane.urmc.rochester.edu/content/wham). Using the following syntax in a terminal

    ./whame min max bins tolerance temp errorAnalysis metadataFile freeEnergyFile
    
will generate the global free energy profile from the different US simulations.

In [ ]:
%%bash
./whame -50 50 100 0.00001 300 0 metadata free

Finally, let's visualize the resulting free energy profile:

In [ ]:
# Extract the free energy
free = np.genfromtxt('free')

# Plot the free energy
plt.plot(free[:,0], free[:,1])
plt.show()

Does this result looks statisfying? How could you obtain a better result? From more and longer simulations performed in the HPC environment, we obtain a result shown belown. Interpret this result compare it with the MTD and US results obtained here.

![alt text](free.png "Cyclohexane free energy profile")

<br/>

Congratulations, you finished the notebook on advanced MD in *Yaff*.